In [4]:
import numpy
import numpy as np


def calcScaleZeroPoint(min_val, max_val, num_bits=8):
    qmin = 0.
    qmax = 2. ** num_bits - 1.
    scale = (max_val - min_val) / (qmax - qmin)

    zero_point = qmax - max_val / scale

    if zero_point < qmin:
        zero_point = qmin
    elif zero_point > qmax:
        zero_point = qmax

    zero_point = int(zero_point)

    return scale, zero_point


def quantize_tensor(x, scale, zero_point, num_bits=8, signed=False):
    if signed:
        qmin = - 2. ** (num_bits - 1)
        qmax = 2. ** (num_bits - 1) - 1
    else:
        qmin = 0.
        qmax = 2. ** num_bits - 1.

    q_x = zero_point + x / scale

    q_x = numpy.clip(q_x, qmin, qmax).round()

    return q_x

def dequantize_tensor(q_x, scale, zero_point):
    return scale * (q_x - zero_point)

In [5]:
import random
import numpy

a = numpy.random.normal(loc=0.0, scale=1.0, size=100000)


In [24]:
k = 1
max_val = a.max()
min_val = a.min()

scale, zero_point = calcScaleZeroPoint(min_val, max_val, k)

scale = 1.5958

print(scale, zero_point)
loss = 0
for i in a:
    q_i = quantize_tensor(i, scale, zero_point, k)
    f_i = dequantize_tensor(q_i, scale, zero_point)
    # print(q_i, i)
    loss += (numpy.abs(f_i - i) ** 2)
loss = loss ** 0.5

print(loss)

1.5958 0
247.2403774020081


In [28]:
import numpy as np

scales = [1.5958, 0.9957, 0.586, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308]

test = np.random.normal(loc=0.22, scale=0.555, size=100000)

def uL2Q(w_f, k):
    if k > 8:
        scale = w_f.max() - w_f.min()
    else:
        scale = scales[k-1]

    a = scale * (np.std(w_f, ddof=1) ** 0.5)
    b = np.mean(w_f)

    fai = (w_f - b) / a - 0.5

    w_q_ = np.clip(fai, -(2**(k-1)), 2**(k-1) - 1).round()

    w_q = a*w_q_ + b

    return w_q_, w_q

w_q_, w_q = uL2Q(test, 2)

# print(w_q_)
# print(test)
# print(w_q)

loss = 0
for i in range(100000):
    loss += (np.abs(w_q[i] - test[i]) ** 2)
loss = loss ** 0.5

print(loss)

135.77853363125806


In [79]:
import torch
from torch import nn

conv2 = nn.Conv2d(3, 64, 3)

def ul2q(w_tensor, num_bits=8):

    if num_bits > 8:
        scale = w_tensor.max() - w_tensor.min()
    else:
        scale = scales[num_bits-1]

    a = scale * (torch.std(w_tensor) ** 0.5)
    b = torch.mean(w_tensor)

    fai = (w_tensor - b) / a - 0.5

    w_q_ = torch.clamp_(fai, -(2**(num_bits-1)), 2**(num_bits-1) - 1).round()

    # w_q_ = torch.tensor(w_q_, dtype=torch.int8)

    w_q = a*w_q_ + b

    return w_q_, w_q



In [80]:
w_q_, w_q = ul2q(conv2.weight.data)

conv2.weight.data = w_q_

test_data = torch.randn(64, 3, 32, 32)

test_data_q, _ = ul2q(test_data)

print(conv2(test_data_q))

tensor([[[[-5.6015e+02,  3.3285e+02,  2.2108e+03,  ..., -1.9532e+03,
           -3.4882e+03,  2.6628e+03],
          [-3.3115e+02,  7.4085e+02, -1.5162e+03,  ..., -4.1515e+02,
           -2.6715e+02, -1.2162e+03],
          [-1.3312e+03,  9.9685e+02, -5.0715e+02,  ...,  1.4768e+03,
            5.9985e+02,  2.0878e+03],
          ...,
          [ 9.0885e+02, -4.3815e+02, -4.9912e+03,  ...,  3.1485e+02,
            1.0588e+03, -5.9015e+02],
          [ 1.4468e+03, -1.0002e+03, -3.3282e+03,  ..., -4.8315e+02,
           -7.8615e+02, -2.5215e+02],
          [-5.7815e+02,  2.2998e+03,  1.1088e+03,  ..., -2.8662e+03,
           -2.7102e+03, -3.7102e+03]],

         [[ 1.2720e+03,  1.8101e+02,  1.8690e+03,  ..., -3.3420e+03,
            2.0101e+02, -1.2870e+03],
          [-2.3690e+03, -6.2399e+02, -6.1299e+02,  ..., -1.0330e+03,
            2.0250e+03, -2.1050e+03],
          [ 1.5980e+03,  6.0801e+02, -8.9999e+02,  ...,  5.9013e+01,
           -3.1399e+02,  8.5601e+02],
          ...,
     